# Homework w06d04 - Natural Language Processing

Load the stumbleupon dataset. It contains the title, the body and the alchemy category of documents. In this exercise, the aim is to use text classification trained on the fetch20_newsgroups dataset to assign one of its classes to each title and body in the stumbleupon dataset. 

This time, the true labels for the stambleupon dataset are unknown, so one has to check if the predictions make sense.  
   
   1. Classify a few texts personally according to the newsgroups classes. 
   
   1. Determine in how many cases the prediction for the body agrees with the prediction for the title. 
   
   1. Check if there is a strong correlation between the alchemy category and the predicted label (at least where the names suggest such a relation).

In [1]:
import numpy as np
import pandas as pd
import json
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer

#### Load the stumbleupon dataset

In [2]:
df = pd.read_csv('stumbleupon.tsv', sep='\t')
df['title'] = df.boilerplate.map(lambda x: json.loads(x).get('title', ''))
df['body'] = df.boilerplate.map(lambda x: json.loads(x).get('body', ''))
df.dropna(inplace=True)
df.head()

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,...,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label,title,body
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,...,24,0,5424,170,8,0.152941,0.079130,0,IBM Sees Holographic Calls Air Breathing Batte...,A sign stands outside the International Busine...
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,...,40,0,4973,187,9,0.181818,0.125448,1,The Fully Electronic Futuristic Starting Gun T...,And that can be carried on a plane without the...
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,...,55,0,2240,258,11,0.166667,0.057613,1,Fruits that Fight the Flu fruits that fight th...,Apples The most popular source of antioxidants...
3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,...,24,0,2737,120,5,0.041667,0.100858,1,10 Foolproof Tips for Better Sleep,There was a period in my life when I had a lot...
4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.719157,2.676471,0.500000,0.222222,0.123457,0.043210,...,14,0,12032,162,10,0.098765,0.082569,0,The 50 Coolest Jerseys You Didn t Know Existed...,Jersey sales is a curious business Whether you...


#### Import the fetch_20newsgroups dataset. Use all categories or a subset.

In [3]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression

data_train = fetch_20newsgroups(subset='train', 
                                categories=fetch_20newsgroups(subset='train').target_names,
                                shuffle=True, random_state=42,
                                remove=('headers', 'footers', 'quotes'))

data_test = fetch_20newsgroups(subset='test', 
                               categories=fetch_20newsgroups(subset='train').target_names,#categories,
                               shuffle=True, random_state=42,
                               remove=('headers', 'footers', 'quotes'))

#### Check which data_train.target_names and which alchemy category could be a good match.

#### Apply a Tfidf Vectorizer to the fetch_20newsgroups training set and train a logistic regression model on it. Check its accuracy on the fetch_20newsgroups test set. 

In [4]:
tf = TfidfVectorizer(stop_words='english',
                     sublinear_tf=True,
                     max_df=0.5,
                     max_features=1000)
tf.fit(data_train['data'])
len(tf.get_feature_names())

1000

In [5]:
model = make_pipeline(TfidfVectorizer(stop_words='english',
                                      sublinear_tf=True,
                                      max_df=0.5,
                                      max_features=1000),
                      LogisticRegression(),
                      )

y_train = data_train['target']
y_test = data_test['target']
model.fit(data_train['data'], y_train)
y_pred = model.predict(data_test['data'])
print accuracy_score(y_test, y_pred)
print "Number of features:", len(model.steps[0][1].get_feature_names())

0.532262347318
Number of features: 1000


#### Now apply a Tfidf Vectorizer to the titles and the body of the stumbleupon dataset and obtain a class prediction for each title and body.

#### Choose three text bodies from the stumbleupon dataset and assign one of the fetch_20newsgroups class labels to it. Does your classification match your prediction?

#### For how many titles and text bodies the prediction agrees?

#### How does the prediction compare with the alchemy category? Check for at least three alchemy categories.